In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [2]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [3]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [4]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [5]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [6]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [7]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=15, batch_size=32, validation_data=(xtest, ytest))

C:\Users\pavan\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 44s 68ms/step - accuracy: 0.3954 - loss: 1.5111 - val_accuracy: 0.7234 - val_loss: 0.8296
Epoch 2/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 28s 70ms/step - accuracy: 0.8768 - loss: 0.4188 - val_accuracy: 0.8031 - val_loss: 0.6074
Epoch 3/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 22s 56ms/step - accuracy: 0.9854 - loss: 0.0594 - val_accuracy: 0.8194 - val_loss: 0.5848
Epoch 4/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - accuracy: 0.9956 - loss: 0.0228 - val_accuracy: 0.8087 - val_loss: 0.6379
Epoch 5/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9968 - loss: 0.0157 - val_accuracy: 0.8091 - val_loss: 0.6919
Epoch 6/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9960 - loss: 0.0172 - val_accuracy: 0.8138 - val_loss: 0.6641
Epoch 7/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 49ms/step - accuracy: 0.9973 - loss: 0.0129 - val_accuracy: 0.8091 - val_loss: 0.7405
Epoch 8/15
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.9973 - loss: 0.0113 - 

In [8]:
model_architecture = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_architecture)

model.save_weights("model_weights.weights.h5")


In [9]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
['sadness']
